In [1]:
#Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
#Import dataset

dataset = pd.read_csv('C:/Users/miant/Desktop/Python/50_Startups.csv')
dataset.head()

R&D Spend  Administration  Marketing Spend       State     Profit
0  165349.20       136897.80        471784.10    New York  192261.83
1  162597.70       151377.59        443898.53  California  191792.06
2  153441.51       101145.55        407934.54     Florida  191050.39
3  144372.41       118671.85        383199.62    New York  182901.99
4  142107.34        91391.77        366168.42     Florida  166187.94

In [3]:
#We define the variables (X and y)

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [4]:
#We need to reshape the variables because the length of "y" must match the length of X

y = y.reshape(len(y),1)

In [5]:
#We need to encode the categorical data --> State

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [3])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[0.0 0.0 1.0 165349.2 136897.8 471784.1]
 [1.0 0.0 0.0 162597.7 151377.59 443898.53]
 [0.0 1.0 0.0 153441.51 101145.55 407934.54]
 [0.0 0.0 1.0 144372.41 118671.85 383199.62]
 [0.0 1.0 0.0 142107.34 91391.77 366168.42]
 [0.0 0.0 1.0 131876.9 99814.71 362861.36]
 [1.0 0.0 0.0 134615.46 147198.87 127716.82]
 [0.0 1.0 0.0 130298.13 145530.06 323876.68]
 [0.0 0.0 1.0 120542.52 148718.95 311613.29]
 [1.0 0.0 0.0 123334.88 108679.17 304981.62]
 [0.0 1.0 0.0 101913.08 110594.11 229160.95]
 [1.0 0.0 0.0 100671.96 91790.61 249744.55]
 [0.0 1.0 0.0 93863.75 127320.38 249839.44]
 [1.0 0.0 0.0 91992.39 135495.07 252664.93]
 [0.0 1.0 0.0 119943.24 156547.42 256512.92]
 [0.0 0.0 1.0 114523.61 122616.84 261776.23]
 [1.0 0.0 0.0 78013.11 121597.55 264346.06]
 [0.0 0.0 1.0 94657.16 145077.58 282574.31]
 [0.0 1.0 0.0 91749.16 114175.79 294919.57]
 [0.0 0.0 1.0 86419.7 153514.11 0.0]
 [1.0 0.0 0.0 76253.86 113867.3 298664.47]
 [0.0 0.0 1.0 78389.47 153773.43 299737.29]
 [0.0 1.0 0.0 73994.56 122782.75 3

In [6]:
#Splitting the data into training and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [7]:
#Feature scaling

from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train = sc_X.fit_transform(X_train)
y_train = sc_y.fit_transform(y_train)

In [8]:
#We start training the SVR

from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train.ravel()) ####

SVR()

In [9]:
#We predict the test results
y_pred = regressor.predict(sc_X.transform(X_test))
y_pred = y_pred.reshape(-1, 1)  # Reshape y_pred to be a 2D array for inverse transform
y_pred = sc_y.inverse_transform(y_pred)  # Now perform the inverse transform

In [10]:
np.set_printoptions(precision=2)
print(np.concatenate((y_pred, y_test.reshape(len(y_test),1)),1))

[[104556.64 103282.38]
 [123328.75 144259.4 ]
 [128933.79 146121.95]
 [ 88368.84  77798.83]
 [143345.55 191050.39]
 [122377.92 105008.31]
 [ 80981.87  81229.06]
 [102957.38  97483.56]
 [119140.36 110352.25]
 [137052.37 166187.94]]


In [11]:
#we predict the test results

#y_pred = sc_y.inverse_transform(regressor.predict(sc_X.transform(X_test)))
#np.set_printoptions(precision=2)
#print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [12]:
#We evaluate the module performance

from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.6574795005932488

# Cross-validation for SVR

In [14]:
from sklearn.model_selection import cross_val_score

In [15]:
sc_X_full = StandardScaler()
sc_y_full = StandardScaler()

In [16]:
X_scaled = sc_X_full.fit_transform(X)
y_scaled = sc_y_full.fit_transform(y.ravel().reshape(-1,1))

In [17]:
#We define the SVR regressor

regressor_sv = SVR(kernel = 'rbf')

In [18]:
#We are ready to perform cross validation with 10 folds

scores = cross_val_score(regressor_sv, X_scaled, y_scaled.ravel(), scoring = 'neg_mean_squared_error', cv= 10)

In [19]:
#We calculate the RMSE for each fold

rmse_scores = np.sqrt(-scores)

In [20]:
#We print/display the results

print("RMSE scores:", rmse_scores)
print("Mean:", rmse_scores.mean())
print("Standard deviation:", rmse_scores.std())

RMSE scores: [1.32 0.24 0.34 0.3  0.2  0.23 0.12 0.31 0.17 1.08]
Mean: 0.43033097501797235
Standard deviation: 0.39336530313525875
